### Master of Applied Artificial Intelligence

**Course: TC5035 - Proyecto Integrador**

<img src="https://github.com/Medicenchapin/Proyecto-Integrador/blob/main/assets/logo.png?raw=1" alt="Image Alt Text" width="500"/>


**Feature engineering**

Tutor: Dr. Horario Martinez Alfaro


Team members:
* Ignacio Jose Aguilar Garcia - A00819762
* Alejandro Calderon Aguilar - A01795353
* Ricardo Mar Cupido - A01795394

# FE

The model contains an individualized FE:

Each row represents a client (or case).

Within each row, in the drivers column, you have something like:

```bash
[
  {"feature": "arpu_90_days", "value": 123.4, "impact": 0.05},
  {"feature": "contacts", "value": 3, "impact": -0.03},
  {"feature": "plan_postpaid", "value": 1, "impact": 0.02},
  ...
]
```

That is, each row has its own important features with different weights.
👉 That's why we can't talk about the same set of global top features at the client level,
because SHAP tells what was most relevant for that particular prediction.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet("../data/campaign_candidates_final.parquet")
df.head()

,state_name,previous_classification,previous_calls,client_age,network_age_years,banking,arpu_90_days,minutes_in,validity_average,average_performance,...,plan_postpaid,sn_banking,digital_index_mean,connected_days,charged_days,apps_days,music_gb,proba,sample_idx,drivers
0,GUATEMALA,NEW CLIENT,0,28.0,3.90,1,214.18,27.93,20.23,0.46,...,0.67,1.00,0.00,91,91,0,5.946,0.714486,0,"[{'feature': 'contacts', 'impact': -0.54814469..."
1,GUATEMALA,NEW CLIENT,0,40.0,0.20,1,135.39,116.22,1.26,0.68,...,0.23,0.46,0.00,67,62,2,0.000,0.620556,6,"[{'feature': 'network_age_years', 'impact': 0...."
2,GUATEMALA,NEW CLIENT,0,19.0,0.66,0,92.58,48.77,3.14,0.61,...,0.75,1.00,0.00,76,55,4,0.031,0.787552,10,"[{'feature': 'contacts', 'impact': -0.42579272..."
3,SAN MARCOS,NEW CLIENT,0,20.0,0.34,1,175.49,83.58,4.00,0.25,...,0.12,0.56,0.00,91,59,30,0.000,0.654747,12,"[{'feature': 'client_age', 'impact': 0.4658642..."
4,SAN MARCOS,NEW CLIENT,0,NaN,1.90,1,97.54,51.89,8.36,0.77,...,0.54,0.38,0.29,90,77,11,0.000,0.708033,26,"[{'feature': 'plan_postpaid', 'impact': 0.5166..."


We have two possible levels of context in the LLM prompts

# 🔹 **Level 1 — Global context prompt**


Includes an overview of the model and what the features represent:



*“The model uses 20 features such as ARPU, previous_calls, plan_postpaid, etc., to predict whether a prepaid user will make a purchase. The following SHAP analysis identifies the most influential features for this user.”*

👉 This gives the semantic model (Ollama or DeepSeek) the context of what the features are.

In [ ]:
def build_global_context(df, feature_playbook=None, top_n=10, rules_text=None):
    """
    Builds a concise, production-ready SYSTEM prompt.
    - df: DataFrame with a 'drivers' column (list[dict]: {'feature','value','impact'})
    - feature_playbook: dict[feature] -> human description (optional)
    - top_n: how many globally most-influential features to list
    - rules_text: optional custom rules block (str). If None, defaults to ARPU band logic.
    """
    # Expandimos todos los drivers en un solo dataframe
    drivers_all = (
        df["drivers"]
        .explode()
        .dropna()
        .apply(pd.Series)  # -> columns: feature, value, impact
    )

    # Importancia global: mean(|impact|)
    global_importance = (
        drivers_all
        .groupby("feature")["impact"]
        .apply(lambda s: s.abs().mean())
        .sort_values(ascending=False)
        .head(top_n)
    )

    top_features = list(global_importance.index)

    # Armamos descripción de features
    lines = []
    for feat in top_features:
        if feature_playbook and feat in feature_playbook:
            desc = feature_playbook[feat]
        else:
            desc = "No description available."
        lines.append(f"- {feat}: {desc}")

    features_block = "\n".join(lines)

    # Default business rules (editable)
    if rules_text is None:
        rules_text = """
        Business Rules (apply consistently):
        1) Window: last 3 full months (M-1, M-2, M-3).
        2) Monthly ARPU = net revenue paid by the customer (top-ups, bundles, add-ons). Exclude freebies/bonuses, chargebacks, and adjustments.
        3) Eligibility: consumption > 0 in each of the 3 months, ARPU_3M_PROM ≥ Q80.00, and no commercial blocks.
        4) Offer mapping by ARPU_3M_PROM:
        • Q80.00–Q110.99 → PLAN_Q115
        • Q111.00–Q130.99 → PLAN_Q135
        • Q131.00–Q155.99 → PLAN_Q160
        • Q156.00–Q180.99 → PLAN_Q185
        • ≥ Q181.00       → PLAN_Q209
        5) Controlled upsell: if ARPU_3M_PROM is in the top 10% of its band and all three monthly ARPUs are ≥ 90% of the next band’s lower bound, offer the next band as an alternative.
        6) Downsell: on price objection, offer the minimum of the current band’s range (do not cross down a band unless affordability constraints are explicit).
        7) Messaging: emphasize benefits, keep price within the assigned band, and anchor value to actual spending.
        """.strip()

    global_prompt = f"""
    You are an expert sales advisor for a prepaid telecom campaign. Maximize conversions with sustainable offers aligned to each customer's real consumption.

    We use a machine learning model trained on historical behavior and engagement to estimate the probability of accepting an offer (sale=1). A higher score means higher likelihood if contacted.

    Most influential features overall (by mean absolute impact):
    {features_block}

    {rules_text}

    Policy:
        - Do NOT reveal internal model weights or math.
    - Do NOT invent personal/sensitive attributes beyond provided data.
    - Keep tone helpful, respectful, and value-focused.
     - You may explain drivers in plain language, but never mention “model”, “probability”, or “SHAP” in the final agent script.
    """.strip()

    return global_prompt


### output

In [9]:
global_prompt = build_global_context(df=df)
print(global_prompt)

You are an expert sales advisor for a prepaid telecom campaign. Maximize conversions with sustainable offers aligned to each customer's real consumption.

    We use a machine learning model trained on historical behavior and engagement to estimate the probability of accepting an offer (sale=1). A higher score means higher likelihood if contacted.

    Most influential features overall (by mean absolute impact):
    - plan_postpaid: No description available.
- music_gb: No description available.
- network_age_years: No description available.
- contacts: No description available.
- client_age: No description available.
- minutes_in: No description available.
- arpu_90_days: No description available.
- charged_days: No description available.
- average_performance: No description available.
- sn_banking: No description available.

    Business Rules (apply consistently):
        1) Window: last 3 full months (M-1, M-2, M-3).
        2) Monthly ARPU = net revenue paid by the customer (top-

## Prompt version 1

```bash
    global_prompt = """
    You are helping generate sales guidance for a prepaid telecom campaign.

    We have a machine learning model that predicts the probability that a customer will accept an offer (sale = 1).
    The model was trained on historical customer behavior and engagement indicators. 
    Higher score means the customer is more likely to buy if contacted.

    The model relies on multiple behavioral and account features. Below are the most influential features overall (averaged across customers), and what they represent:

    {features_block}

    Rules:
    - NEVER reveal internal model weights or math details.
    - NEVER invent personal/sensitive attributes not present in the features.
    - Keep tone helpful, respectful, and focused on value to customer.
    - You are allowed to explain *why the model thinks a segment is likely to buy*, in plain language.
        """.strip()
```

### prompt version 2

```bash
    global_prompt = f"""
    You are an expert sales advisor for a prepaid telecom campaign. Maximize conversions with sustainable offers aligned to each customer's real consumption.

    We use a machine learning model trained on historical behavior and engagement to estimate the probability of accepting an offer (sale=1). A higher score means higher likelihood if contacted.

    Most influential features overall (by mean absolute impact):
    {features_block}

    {rules_text}

    Policy:
        - Do NOT reveal internal model weights or math.
    - Do NOT invent personal/sensitive attributes beyond provided data.
    - Keep tone helpful, respectful, and value-focused.
     - You may explain drivers in plain language, but never mention “model”, “probability”, or “SHAP” in the final agent script.
    """.strip()
```

# 🔹 **Level 2 — Row-specific prompt (customized by customer)**


Here it uses the customer's SHAP drivers (their relevant features, values, and impacts):

In [ ]:
def build_customer_prompt(row, driver_list, extra_context_cols=None, name_field=None):
    """
    row: una fila de tu df (por ejemplo df.loc[idx])
         que tiene columnas humanas tipo 'state_name', 'previous_classification', etc.
         y también 'proba'.

    driver_list: lista de dicts tipo:
      [
        {"feature": "arpu_90_days", "value": 8.24, "impact": 0.32},
        ...
      ]

    extra_context_cols: lista de columnas del row que quieres pasar al LLM
                        para que hable más personalizado.
                        Ej: ["state_name", "previous_classification", "arpu_90_days", "network_age_years"]
    """

    # 1. Prepara resumen de los drivers SHAP personalizados de este cliente
    driver_lines = []
    for d in driver_list:
        driver_lines.append(
            f"- {d['feature']}: value={d['value']}, impact={d['impact']:+.3f}"
        )
    driver_block = "\n".join(driver_lines)

    # 2. Agrega contexto humano del cliente (opcional)
    context_lines = []
    if extra_context_cols:
        for col in extra_context_cols:
            if col in row:
                context_lines.append(f"{col} = {row[col]}")
    context_block = "\n".join(context_lines) if context_lines else "No additional context."
    
    # Optional name for sample script
    name_value = (row.get(name_field) if name_field and name_field in row else "Customer")

    # 3. Construye el prompt final
    prompt = f"""
    [Customer Context]
    - Acceptance likelihood (score): {row.get('proba', float('nan')):.2%}
    - Attributes:
    {context_block}

    - Top influencing factors for this specific customer:
    {driver_block}

    [Task]
    Using ONLY the context above and the campaign rules from system prompt:
    1) Decide Eligibility: {{Yes/No}} and give a brief reason if "No".
    2) Select Suggested Band/Plan: {{PLAN_Q115|PLAN_Q135|PLAN_Q160|PLAN_Q185|PLAN_Q209}}.
    3) Provide Authorized offer range: {{Qxx.xx–Qyy.yy}}.
    4) Recommend an initial price within the authorized range: {{Qxx.xx}}. Justify in one line referencing recent spend.
    5) If applicable, propose an Upsell option (next band) with a one-line justification.

    Then produce a concise 3-line agent script:
    - Value: tie benefits to recent spend (“with what you already invest per month…”).
    - Price: keep within the assigned range.
    - Close: immediate activation, no contract, same line/top-ups.

    [Output Format]
    Eligibility: <Yes/No> (+ reason if No)
    Suggested Plan: <PLAN_Q115|PLAN_Q135|PLAN_Q160|PLAN_Q185|PLAN_Q209>
    Authorized range: <Qxx.xx–Qyy.yy>
    Recommended price: <Qxx.xx>  # one-line justification
    Upsell option: <plan_if_any>  # brief justification
    Script:
    “{name_value}, over the last 3 months you’ve invested about Q<arpu_3m_prom>/month.
    With the <plan_sugerido> plan you get more data/minutes for Q<precio_recomendado>, keeping your usual spend but with more value.
    Shall I confirm activation? It goes live today — no contract, and you keep your same line.”
    """.strip()

    return prompt


### output

In [6]:
idx = 42
row = df.loc[idx]

prompt_for_customer = build_customer_prompt(
    row=row,
    driver_list=row["drivers"],
    extra_context_cols=[
        "state_name",
        "previous_classification",
        "arpu_90_days",
        "network_age_years",
        "connected_days",
    ]
)

print(prompt_for_customer)

We are preparing a telemarketing/sales pitch for a prepaid mobile customer.

    Predicted probability of accepting the offer: 64.30%

    Relevant context for this customer:
    state_name = CHIQUIMULA
previous_classification = NEW CLIENT
arpu_90_days = 99.49
network_age_years = 3.78
connected_days = 77

    The following factors were most influential in predicting that this customer is likely to accept an offer:
    - plan_postpaid: value=1.5579612144082289, impact=+0.490
- contacts: value=-1.6741857759106464, impact=-0.398
- arpu_90_days: value=-0.9999859292761342, impact=+0.178
- client_age: value=-0.0363905226932068, impact=-0.112
- start_using_months: value=-0.5718190794140261, impact=+0.108

    Task:
    1. Explain, in plain language, why this customer might respond positively.
    2. Suggest how an agent should position the offer (tone, focus, what to mention).
    3. Keep it short and actionable, as guidance for a call center agent.
    4. Do NOT mention 'model', 'probability

### Prompt version 1

```bash
    prompt = f"""
    The model predicts a sale probability of {proba:.2%} for this prepaid customer.
    The most influential factors for this prediction were:
    {summary}

    Explain, in natural language, why these features may lead to this prediction.
    Provide a concise and interpretable summary.
    """
```

### prompt version 2

```bash
    prompt = f"""
    We are preparing a telemarketing/sales pitch for a prepaid mobile customer.

    Predicted probability of accepting the offer: {row['proba']:.2%}

    Relevant context for this customer:
    {context_block}

    The following factors were most influential in predicting that this customer is likely to accept an offer:
    {driver_block}

    Task:
    1. Explain, in plain language, why this customer might respond positively.
    2. Suggest how an agent should position the offer (tone, focus, what to mention).
    3. Keep it short and actionable, as guidance for a call center agent.
    4. Do NOT mention 'model', 'probability', 'algorithm', 'prediction', or 'SHAP'. Just speak as advice.
    """.strip()
```